In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
X_data = pd.read_csv("X_data.csv")
X_data = X_data.to_numpy(dtype=np.float32)

In [3]:
Y_data = pd.read_csv("Y_data.csv")
Y_data = Y_data.to_numpy(dtype=np.float32)

Пусть размерность вектора $x_1 - n_1$, $x_2 - n_2$ и $x_3 - n_3$

В случае нашей выборки #3, $n_1 = 2$, $n_2 = 2$ и $n_3 = 2$

In [4]:
n1, n2, n3 = 2, 2, 2

In [5]:
X1 = X_data[:, 0: n1]
X2 = X_data[:, n1: n1 + n2]
X3 = X_data[:, n1 + n2: n1 + n2 + n3]

In [6]:
def norm_all_matrix(a):
    min_element = a.min()
    max_element = a.max()
    result = copy.deepcopy(a)
    result = result.astype(np.float32)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            result[i, j] = (result[i, j] - min_element) / (max_element - min_element)
    
    return result    

In [7]:
X1_norm = norm_all_matrix(X1)
X2_norm = norm_all_matrix(X2)
X3_norm = norm_all_matrix(X3)
Y_norm = norm_all_matrix(Y_data)

In [8]:
def chebyshev_polynom_in_point(x, degree):
    values = np.zeros(degree + 1)
    values[0] = 1
    values[1] = x
    for current_degree in range(2, degree + 1):
        values[current_degree] = 2 * x * values[current_degree - 1] - values[current_degree - 2]
    
    return values[degree]

In [9]:
def star_chebyshev_polynom_in_point(x, degree):
    if degree == 0:
        return 0.5
    else:
        return chebyshev_polynom_in_point(2 * x - 1, degree)

Для начала зафиксируем степень смещенных полиномов Чебешева $T_{P_1}^{*}(x)$, $T_{P_2}^{*}(x)$ и $T_{P_3}^{*}(x)$:

In [10]:
P1, P2, P3 = 3, 3, 3

Составим систему уравнений и решим ее для нахождения $||\lambda||$

In [11]:
def find_lambdas(X, Y, degrees_of_polynomials, size_of_vectors, calc_polynom_in_point):
    A = []
    N = Y.shape[0]
    for sample_number in range(N):
        current_row = []
        for i, X_i in enumerate(X):
            current_row = np.hstack([current_row, [calc_polynom_in_point(X_i[sample_number][j], degree)
                                                   for j in range(size_of_vectors[i])
                                                   for degree in range(degrees_of_polynomials[i] + 1)
                                                  ]
                                    ])
        A.append(current_row)
    
    lambdas = [np.linalg.lstsq(A, Y[:, k])[0] for k in range(Y.shape[1])]
    return lambdas

In [12]:
lambdas = find_lambdas([X1_norm, X2_norm, X3_norm], Y_norm, 
                       [P1, P2, P3], [n1, n2, n3], 
                       star_chebyshev_polynom_in_point)

/home/vladislav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


Теперь составим три системы для нахождение $||\alpha||$

In [13]:
def find_alphas(X, Y, degrees_of_polynomials, size_of_vectors, calc_polynom_in_point, lambdas):
    alphas = []
    phi_count = Y.shape[1]
    N = Y.shape[0]
    for phi_idx in range(phi_count):
        B = []
        for i, X_i in enumerate(X):
            current_B = []
            for sample_number in range(N):
                current_row = []
                idx = 0
                for z in range(i):
                    idx += size_of_vectors[z] * (degrees_of_polynomials[z] + 1)
                for j in range(size_of_vectors[i]):
                    c = 0
                    for degree in range(degrees_of_polynomials[i] + 1):
                        c += calc_polynom_in_point(X_i[sample_number][j], degree) * lambdas[phi_idx][idx]
                        idx += 1
                    current_row.append(c)
                current_B.append(current_row)
            B.append(current_B)
        alphas.append([np.linalg.lstsq(A, Y[:, phi_idx])[0] for A in B])
    
    return alphas

In [14]:
alphas = find_alphas([X1_norm, X2_norm, X3_norm], Y_norm, 
                                     [P1, P2, P3], [n1, n2, n3], 
                                     star_chebyshev_polynom_in_point,
                                     lambdas)

/home/vladislav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [23]:
coeff1, coeff2, coeff3 = alphas[0]

In [24]:
phi_number = 0

Теперь составим и решим систему для нахождения коэффициентов $||c||$

In [25]:
def find_C(X, Y, degrees_of_polynomials, size_of_vectors, calc_polynom_in_point, lambdas, alphas):
    C = []
    phi_count = Y.shape[1]
    N = Y.shape[0]
    for phi_idx in range(phi_count):
        current_C = []
        for sample_number in range(N):
            coeff = []
            for i, X_i in enumerate(X):
                current_coeff = 0
                idx = 0
                for z in range(i):
                    idx += size_of_vectors[z] * (degrees_of_polynomials[z] + 1)
                for j in range(size_of_vectors[i]):
                    for degree in range(degrees_of_polynomials[i] + 1):
                        current_coeff += alphas[phi_idx][i][j] * lambdas[phi_idx][idx] * calc_polynom_in_point(X_i[sample_number][j], degree)
                        idx += 1
                coeff.append(current_coeff)
            current_C.append(coeff)
        C.append(current_C)
        
    result = [np.linalg.lstsq(C[k], Y[:, k])[0] for k in range(phi_count)]
    return result

In [26]:
final_coeff = find_C([X1_norm, X2_norm, X3_norm], Y_norm, 
                                     [P1, P2, P3], [n1, n2, n3], 
                                     star_chebyshev_polynom_in_point,
                                     lambdas, alphas)[0]

/home/vladislav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [27]:
def calc(x1, x2, x3):
    result = 0
    
    idx = 0
    for j1 in range(n1):
        for p1 in range(P1 + 1):
            cur_coeff = coeff1[j1] * lambdas[phi_number][idx]
            idx += 1
            result += star_chebyshev_polynom_in_point(x1[j1], p1) * cur_coeff * final_coeff[0]

    idx = n1 * (P1 + 1)
    for j2 in range(n2):
        for p2 in range(P2 + 1):
            cur_coeff = coeff2[j2] * lambdas[phi_number][idx]
            idx += 1
            result += star_chebyshev_polynom_in_point(x2[j2], p2) * cur_coeff * final_coeff[1]
    
    idx = n1 * (P1 + 1) + n2 * (P2 + 1)
    for j3 in range(n3):
        for p3 in range(P3 + 1):
            cur_coeff = coeff3[j3] * lambdas[phi_number][idx]
            idx += 1
            result += star_chebyshev_polynom_in_point(x3[j3], p3) * cur_coeff * final_coeff[2]

    return result

In [28]:
def calc_norm(x1, x2, x3):
    return calc((x1 - X1.min()) / (X1.max() - X1.min()),
                (x2 - X2.min()) / (X2.max() - X2.min()),
                (x3 - X3.min()) / (X3.max() - X3.min())) * (Y_data.max() - Y_data.min()) + Y_data.min()

In [29]:
def mse_error():
    result = 0
    for i in range(40):
        result += abs(calc_norm(X1[i], X2[i], X3[i]) - Y_data[i, phi_number])
    
    return result / 40

In [30]:
print(mse_error())

0.23187315989241428


In [31]:
phi_count = Y_norm.shape[1]

In [32]:
result = find_C([X1_norm, X2_norm, X3_norm], Y_norm, 
                                     [P1, P2, P3], [n1, n2, n3], 
                                     star_chebyshev_polynom_in_point,
                                     lambdas, alphas)

/home/vladislav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Выведим разложение $\Phi_1$, $\Phi_2$, $\Phi_3$, $\Phi_4$ и $\Phi_5$

In [33]:
for phi_idx in range(phi_count):
    print("Ф%d(x1, x2, x3) = %.4f * Ф%d1(x1, x2, x3) + %.4f * Ф%d2(x1, x2, x3) + %.4f * Ф%d3(x1, x2, x3)" % (phi_idx + 1, result[phi_idx][0], 
                                                                                                                phi_idx + 1, result[phi_idx][1],
                                                                                                                phi_idx + 1, result[phi_idx][2], phi_idx + 1))

    print("")

Ф1(x1, x2, x3) = 0.4697 * Ф11(x1, x2, x3) + 0.0569 * Ф12(x1, x2, x3) + 0.5731 * Ф13(x1, x2, x3)

Ф2(x1, x2, x3) = 0.5547 * Ф21(x1, x2, x3) + 0.1518 * Ф22(x1, x2, x3) + 0.6786 * Ф23(x1, x2, x3)

Ф3(x1, x2, x3) = 0.4644 * Ф31(x1, x2, x3) + 0.0542 * Ф32(x1, x2, x3) + 0.5599 * Ф33(x1, x2, x3)

Ф4(x1, x2, x3) = 0.4534 * Ф41(x1, x2, x3) + 0.0564 * Ф42(x1, x2, x3) + 0.5117 * Ф43(x1, x2, x3)

Ф5(x1, x2, x3) = 0.5076 * Ф51(x1, x2, x3) + 0.0898 * Ф52(x1, x2, x3) + 0.6324 * Ф53(x1, x2, x3)



In [34]:
degrees = [P1, P2, P3]
sizes = [n1, n2, n3]
phi_count = Y_norm.shape[1]

In [35]:
for phi_idx in range(phi_count):
    answer = "Ф%d(x1, x2, x3) = " % (phi_idx + 1)
    for num_x in [0, 1, 2]:
        idx = 0
        for z in range(num_x):
            idx += sizes[z] * (degrees[z] + 1)
        for j in range(sizes[num_x]):
            for deg in range(degrees[num_x] + 1):
                answer += "+ %.4f * T%d(x%d)" % (alphas[phi_idx][num_x][j] * lambdas[phi_idx][idx] * result[phi_idx][num_x], deg, (num_x + 1) * 10 + j + 1)
                idx += 1
            answer += "\n"
    print(answer)

Ф1(x1, x2, x3) = + 0.3588 * T0(x11)+ 0.1319 * T1(x11)+ 0.0000 * T2(x11)+ 0.0000 * T3(x11)
+ 0.1458 * T0(x12)+ 0.1206 * T1(x12)+ -0.0000 * T2(x12)+ -0.0000 * T3(x12)
+ 0.0327 * T0(x21)+ 0.0057 * T1(x21)+ -0.0000 * T2(x21)+ -0.0000 * T3(x21)
+ 0.0134 * T0(x22)+ 0.0057 * T1(x22)+ 0.0012 * T2(x22)+ -0.0000 * T3(x22)
+ 0.1302 * T0(x31)+ 0.1447 * T1(x31)+ 0.0275 * T2(x31)+ -0.0000 * T3(x31)
+ 0.3744 * T0(x32)+ 0.1032 * T1(x32)+ -0.0000 * T2(x32)+ 0.0000 * T3(x32)

Ф2(x1, x2, x3) = + 0.1756 * T0(x11)+ 0.1010 * T1(x11)+ -0.0000 * T2(x11)+ -0.0000 * T3(x11)
+ 0.0722 * T0(x12)+ 0.0934 * T1(x12)+ 0.0000 * T2(x12)+ -0.0000 * T3(x12)
+ 0.0318 * T0(x21)+ 0.0087 * T1(x21)+ 0.0000 * T2(x21)+ 0.0000 * T3(x21)
+ 0.0178 * T0(x22)+ 0.0118 * T1(x22)+ 0.0025 * T2(x22)+ 0.0000 * T3(x22)
+ 0.0767 * T0(x31)+ 0.1335 * T1(x31)+ 0.0254 * T2(x31)+ -0.0000 * T3(x31)
+ 0.1681 * T0(x32)+ 0.0725 * T1(x32)+ 0.0000 * T2(x32)+ -0.0000 * T3(x32)

Ф3(x1, x2, x3) = + 0.1950 * T0(x11)+ 0.0659 * T1(x11)+ -0.0000 * T2(x11)+ -0

In [36]:
class Polynom:
    def __init__(self, deg, coeff):
        self.deg = deg
        self.coeff = coeff
        
    def sum(self, rhs):
        result = Polynom(max(rhs.deg, self.deg), np.zeros(max(rhs.deg, self.deg) + 1))
        coeff1 = rhs.coeff[::-1]
        coeff2 = self.coeff[::-1]
        for i in range(max(rhs.deg, self.deg) + 1):
            cand1 = 0
            if i <= rhs.deg:
                cand1 = coeff1[i]
            
            cand2 = 0
            if i <= self.deg:
                cand2 = coeff2[i]
            
            result.coeff[i] = cand1 + cand2
    
        result.coeff = result.coeff[::-1]
        return result
    
    def mul(self, rhs):
        result = Polynom(rhs.deg + self.deg, np.zeros(rhs.deg + self.deg + 1))
        coeff1 = rhs.coeff[::-1]
        coeff2 = self.coeff[::-1]
        result_coeff = np.zeros(rhs.deg + self.deg + 1)
        for deg1, val1 in enumerate(coeff1):
            for deg2, val2 in enumerate(coeff2):
                result_coeff[deg1 + deg2] += val1 * val2
        
        result.coeff = result_coeff[::-1]
        
        return result
    
    def mul_for_const(self, k):
        result = Polynom(self.deg, copy.deepcopy(self.coeff))
        for i in range(self.deg + 1):
            result.coeff[i] = result.coeff[i] * k
        return result
    
    def calc(self, point):
        result = 0
        for deg, coeff in enumerate(self.coeff[::-1]):
            result += coeff * (point ** deg)
        return result
    
    def factorial(n):
        if n == 0 or n == 1:
            return 1
        
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result
    
    def cnk(n, k):
        return Polynom.factorial(n) / (Polynom.factorial(n - k) * Polynom.factorial(k))
    
    def calc_binom(a, b, n):
        result = Polynom(0, [0])
        for k in range(n + 1):
            c = Polynom.cnk(n, k) * (a ** k) * (b ** (n - k))
            coeff_arr = np.zeros(k + 1)
            coeff_arr[0] = c
            result = result.sum(Polynom(k, coeff_arr))
        
        return result
        
    def substitution(self, a, b):
        result = Polynom(0, [0])
        for deg, coeff in enumerate(self.coeff[::-1]):
            result = result.sum(Polynom.calc_binom(a, b, deg).mul_for_const(coeff))
        return result
    
    def add_const(self, k):
        result = Polynom(self.deg, copy.deepcopy(self.coeff))
        result.coeff[-1] += k
        return result
        
    
    def print(self, name='x'):
        answer = ""
        for idx, coeff in enumerate(self.coeff):
            if abs(coeff) > 1e-6:
                if idx != self.deg:
                    answer += "%.6f * %s ^ %d + " % (coeff, name, self.deg - idx)
                else:
                    answer += "%.6f" % (coeff)
        print(answer)

In [39]:
def chebyshev_polynom(degree):
    if degree == 0:
        return Polynom(0, [1])
    values = [0] * (degree + 1)
    values[0] = Polynom(0, [1])
    values[1] = Polynom(1, [1, 0])
    for current_degree in range(2, degree + 1):
        values[current_degree] = values[current_degree - 1].mul(Polynom(1, [2, 0])).sum(values[current_degree - 2].mul_for_const(-1))
    return values[degree]

In [40]:
def star_chebyshev_polynom(degree):
    if degree == 0:
        return Polynom(0, [0.5])
    values = [0] * (degree + 1)
    values[0] = Polynom(0, [1])
    values[1] = Polynom(1, [2, -1])
    for current_degree in range(2, degree + 1):
        values[current_degree] = values[current_degree - 1].mul(Polynom(1, [4, -2])).sum(values[current_degree - 2].mul_for_const(-1))
    return values[degree]

Полиномы для нормированных данных

In [44]:
for phi_idx in range(phi_count):
    check = 0
    print("Ф%d(x1, x2, x3) = " % (phi_idx + 1), end='')
    for num_x in [0, 1, 2]:
        idx = 0
        for z in range(num_x):
            idx += sizes[z] * (degrees[z] + 1)
        for j in range(sizes[num_x]):
            current_polynom = Polynom(0, [0])
            for deg in range(degrees[num_x] + 1):
                current_polynom = current_polynom.sum(star_chebyshev_polynom(deg).mul_for_const(alphas[phi_idx][num_x][j] * lambdas[phi_idx][idx] * result[phi_idx][num_x]))
                idx += 1

            current_polynom.print(name='x%d' % ((num_x + 1) * 10 + j + 1))

    print()

Ф1(x1, x2, x3) = 0.000031 * x11 ^ 3 + -0.000036 * x11 ^ 2 + 0.263785 * x11 ^ 1 + 0.047535
-0.000002 * x12 ^ 3 + -0.000002 * x12 ^ 2 + 0.241119 * x12 ^ 1 + -0.047664
-0.000001 * x21 ^ 3 + 0.011414 * x21 ^ 1 + 0.010639
0.009816 * x22 ^ 2 + 0.001643 * x22 ^ 1 + 0.002222
-0.000001 * x31 ^ 3 + 0.220073 * x31 ^ 2 + 0.069374 * x31 ^ 1 + -0.052128
0.000010 * x32 ^ 3 + -0.000015 * x32 ^ 2 + 0.206385 * x32 ^ 1 + 0.083989

Ф2(x1, x2, x3) = -0.000002 * x11 ^ 3 + 0.000002 * x11 ^ 2 + 0.201972 * x11 ^ 1 + -0.013182
-0.000002 * x12 ^ 3 + 0.000003 * x12 ^ 2 + 0.186846 * x12 ^ 1 + -0.057322
0.017378 * x21 ^ 1 + 0.007216
0.020270 * x22 ^ 2 + 0.003395 * x22 ^ 1 + -0.000422
0.202969 * x31 ^ 2 + 0.063977 * x31 ^ 1 + -0.069738
-0.000004 * x32 ^ 3 + 0.000006 * x32 ^ 2 + 0.144996 * x32 ^ 1 + 0.011548

Ф3(x1, x2, x3) = -0.000023 * x11 ^ 3 + 0.000027 * x11 ^ 2 + 0.131757 * x11 ^ 1 + 0.031619
0.000001 * x12 ^ 3 + 0.000001 * x12 ^ 2 + 0.122051 * x12 ^ 1 + -0.020887
0.005648 * x21 ^ 1 + 0.005974
0.004722 * x22 ^ 2

Полиномы для исходных данных

In [45]:
for phi_idx in range(phi_count):
    check = 0
    const_add_one_time = False
    print("Ф%d(x1, x2, x3) = " % (phi_idx + 1), end='')
    for num_x in [0, 1, 2]:
        idx = 0
        for z in range(num_x):
            idx += sizes[z] * (degrees[z] + 1)
        for j in range(sizes[num_x]):
            current_polynom = Polynom(0, [0])
            for deg in range(degrees[num_x] + 1):
                temp_polynom = star_chebyshev_polynom(deg).mul_for_const(alphas[phi_idx][num_x][j] * lambdas[phi_idx][idx] * result[phi_idx][num_x])
                idx += 1
                if num_x == 0:
                    temp_polynom = temp_polynom.substitution(1 / (X1.max() - X1.min()), -X1.min() / (X1.max() - X1.min()))
                elif num_x == 1:
                    temp_polynom = temp_polynom.substitution(1 / (X2.max() - X2.min()), -X2.min() / (X2.max() - X2.min()))
                else:
                    temp_polynom = temp_polynom.substitution(1 / (X3.max() - X3.min()), -X3.min() / (X3.max() - X3.min()))
                
                
                current_polynom = current_polynom.sum(temp_polynom)
            
            current_polynom = current_polynom.mul_for_const(Y_data.max() - Y_data.min())    
            if not const_add_one_time:
                const_add_one_time = True
                current_polynom = current_polynom.add_const(Y_data.min())
                

            current_polynom.print(name='x%d' % ((num_x + 1) * 10 + j + 1))
    print()

Ф1(x1, x2, x3) = 0.000023 * x11 ^ 3 + -0.000054 * x11 ^ 2 + 0.785995 * x11 ^ 1 + 0.996558
-0.000001 * x12 ^ 3 + -0.000003 * x12 ^ 2 + 0.718457 * x12 ^ 1 + -0.284049
0.000001 * x21 ^ 2 + 0.035802 * x21 ^ 1 + 0.063399
0.016204 * x22 ^ 2 + 0.005154 * x22 ^ 1 + 0.013245
0.327874 * x31 ^ 2 + 0.206714 * x31 ^ 1 + -0.310648
0.000007 * x32 ^ 3 + -0.000022 * x32 ^ 2 + 0.614962 * x32 ^ 1 + 0.500522

Ф2(x1, x2, x3) = -0.000002 * x11 ^ 3 + 0.000003 * x11 ^ 2 + 0.601812 * x11 ^ 1 + 0.634726
-0.000001 * x12 ^ 3 + 0.000004 * x12 ^ 2 + 0.556741 * x12 ^ 1 + -0.341601
0.054505 * x21 ^ 1 + 0.043002
0.033462 * x22 ^ 2 + 0.010648 * x22 ^ 1 + -0.002515
0.302392 * x31 ^ 2 + 0.190631 * x31 ^ 1 + -0.415594
-0.000003 * x32 ^ 3 + 0.000009 * x32 ^ 2 + 0.432043 * x32 ^ 1 + 0.068821

Ф3(x1, x2, x3) = -0.000017 * x11 ^ 3 + 0.000040 * x11 ^ 2 + 0.392594 * x11 ^ 1 + 0.901707
0.000002 * x12 ^ 2 + 0.363672 * x12 ^ 1 + -0.124473
0.017714 * x21 ^ 1 + 0.035603
0.007795 * x22 ^ 2 + 0.002482 * x22 ^ 1 + 0.008067
0.161494 * x